In [1]:
import pathlib
import datetime
import os

In [2]:
import iris
import iris.cube
import cf_units

In [3]:
import pandas

In [4]:
import xarray

In [5]:
import matplotlib
import matplotlib.pyplot

In [6]:
data_root = pathlib.Path('/scratch/shaddad/precip_rediagnosis/train_202207/')
event_name = '202110_nswws_amber_oct'
event_dir = data_root / event_name

In [7]:
mg_dir = event_dir / 'mogreps-g'
radar_dir = event_dir / 'radar' 

In [8]:
event_dir

PosixPath('/scratch/shaddad/precip_rediagnosis/train_202207/202110_nswws_amber_oct')

In [9]:
validity_times = pandas.date_range(
    start=datetime.datetime(2021,10,19,18,0), 
    end=datetime.datetime(2021,10,21,0,0), 
    freq=datetime.timedelta(hours=6),
).to_pydatetime()
validity_times

array([datetime.datetime(2021, 10, 19, 18, 0),
       datetime.datetime(2021, 10, 20, 0, 0),
       datetime.datetime(2021, 10, 20, 6, 0),
       datetime.datetime(2021, 10, 20, 12, 0),
       datetime.datetime(2021, 10, 20, 18, 0),
       datetime.datetime(2021, 10, 21, 0, 0)], dtype=object)

In [10]:
radar_df_path = [p1 for p1 in radar_dir.iterdir() if 'csv' in str(p1) and 'prd_radar' in str(p1)][0]
radar_nc_path = [p1 for p1 in radar_dir.iterdir() if 'nc' in str(p1) and 'prd_radar' in str(p1)][0]
mg_df_path = [p1 for p1 in mg_dir.iterdir() if 'csv' in str(p1) and 'prd_mogreps' in str(p1)][0]


In [11]:
mg_df = pandas.read_csv(mg_df_path)
mg_df

,realization,latitude,longitude,forecast_period,forecast_reference_time,time,cloud_area_fraction,rainfall_rate,lwe_snowfall_rate,surface_altitude,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,0,49.40625,-5.484375,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,1.991540,0.0,0.0,...,22.8125,23.2500,23.5625,24.0000,24.5000,25.0625,26.4375,28.5625,30.6875,32.3750
1,0,49.40625,-5.203125,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,2.058595,0.0,0.0,...,21.0625,21.4375,21.9375,22.6250,23.3750,24.1875,26.0625,28.0000,30.0000,31.5000
2,0,49.40625,-4.921875,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,3.144890,0.0,0.0,...,18.8125,19.2500,19.8750,20.6250,21.5000,22.5000,25.1250,27.8750,30.4375,32.2500
3,0,49.40625,-4.640625,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,3.557280,0.0,0.0,...,19.1875,19.6875,20.3125,21.0000,21.7500,22.7500,25.4375,28.8750,32.0000,33.9375
4,0,49.40625,-4.359375,0 days 06:00:00,2021-10-19 12:00:00,2021-10-19 18:00:00,1.000000,1.756847,0.0,0.0,...,22.0625,22.6250,23.3125,24.0625,24.8125,25.7500,27.9375,31.5000,34.4375,36.1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192775,17,58.78125,1.546875,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.921875,0.000000,0.0,0.0,...,15.5000,14.9375,14.5000,14.1250,13.8750,13.7500,14.5000,14.6875,14.8125,15.8750
192776,17,58.78125,1.828125,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.937500,0.000000,0.0,0.0,...,15.3750,14.7500,14.1875,13.6875,13.1875,12.9375,13.4375,13.3750,14.0000,15.4375
192777,17,58.78125,2.109375,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.531250,0.000000,0.0,0.0,...,15.5625,14.8125,14.1250,13.5000,12.8750,12.5625,12.6875,12.4375,12.9375,14.7500
192778,17,58.78125,2.390625,0 days 06:00:00,2021-10-21,2021-10-21 06:00:00,0.906250,0.000000,0.0,0.0,...,15.8125,15.0625,14.2500,13.5000,12.9375,12.6875,12.6250,12.0625,12.0625,13.9375


In [18]:
mg_df[mg_df['time'] == '2021-10-20']['realization'].value_counts()

26    1530
33    1530
19    1530
20    1530
21    1530
22    1530
23    1530
24    1530
25    1530
18    1530
27    1530
28    1530
29    1530
30    1530
31    1530
32    1530
34    1318
0     1163
Name: realization, dtype: int64

In [17]:
mg_df[mg_df['time'] == '2021-10-20 00:00:00']['realization'].value_counts()

0     367
34    212
Name: realization, dtype: int64

In [19]:
mg_df[mg_df['time'] == '2021-10-20 06:00:00']['realization'].value_counts()

0     1530
1     1530
16    1530
15    1530
14    1530
13    1530
12    1530
11    1530
10    1530
9     1530
8     1530
7     1530
6     1530
5     1530
4     1530
3     1530
2     1530
17    1530
Name: realization, dtype: int64

In [42]:
df2 = pandas.DataFrame(mg_df[mg_df['time'] == '2021-10-21'])
df2

,realization,latitude,longitude,forecast_period,forecast_reference_time,time,cloud_area_fraction,rainfall_rate,lwe_snowfall_rate,surface_altitude,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
138116,0,51.84375,1.828125,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.921875,1.743436,0.0,0.0,...,12.2500,12.1875,12.0625,11.9375,11.7500,11.5000,10.3750,9.0625,7.5625,6.0625
138117,0,51.84375,2.109375,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.406250,0.000000,0.0,0.0,...,15.6875,14.7500,13.8125,12.8750,12.0000,11.0625,9.5625,8.1250,7.0000,6.0000
138118,0,51.84375,2.390625,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.296875,0.000000,0.0,0.0,...,16.8125,15.8125,14.8750,13.8750,12.8750,11.9375,10.4375,9.1875,8.2500,7.5625
138119,0,51.84375,2.671875,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.359375,0.000000,0.0,0.0,...,18.1250,17.3125,16.5000,15.5625,14.6875,13.8125,12.5625,11.5625,10.6875,10.1250
138120,0,52.03125,-5.484375,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.296875,0.000000,0.0,0.0,...,15.6875,16.0625,16.5000,17.0625,17.6875,18.3750,18.1250,17.1250,17.5625,19.1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165072,34,57.84375,-2.109375,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.546875,0.124052,0.0,0.0,...,19.7500,19.5625,19.3750,19.1875,18.9375,18.6250,18.3125,17.3125,17.5000,19.3750
165073,34,57.84375,-1.828125,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.562500,0.013411,0.0,0.0,...,19.0000,18.8125,18.6250,18.5000,18.3750,18.2500,17.8750,16.8125,17.9375,20.3750
165074,34,57.84375,-1.546875,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.500000,0.026822,0.0,0.0,...,18.1875,18.0625,17.9375,17.8750,17.8750,17.8750,17.3125,16.3750,18.7500,21.8750
165075,34,57.84375,-1.265625,0 days 06:00:00,2021-10-20 18:00:00,2021-10-21,0.531250,0.013411,0.0,0.0,...,17.5000,17.3750,17.3750,17.4375,17.5625,17.5625,16.4375,16.1250,20.2500,24.1250


In [53]:
df2['time'] = pandas.to_datetime([datetime.datetime(2021,10,23,0,0,0)] * df2.shape[0])
df2

,realization,latitude,longitude,forecast_period,forecast_reference_time,time,cloud_area_fraction,rainfall_rate,lwe_snowfall_rate,surface_altitude,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
138116,0,51.84375,1.828125,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.921875,1.743436,0.0,0.0,...,12.2500,12.1875,12.0625,11.9375,11.7500,11.5000,10.3750,9.0625,7.5625,6.0625
138117,0,51.84375,2.109375,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.406250,0.000000,0.0,0.0,...,15.6875,14.7500,13.8125,12.8750,12.0000,11.0625,9.5625,8.1250,7.0000,6.0000
138118,0,51.84375,2.390625,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.296875,0.000000,0.0,0.0,...,16.8125,15.8125,14.8750,13.8750,12.8750,11.9375,10.4375,9.1875,8.2500,7.5625
138119,0,51.84375,2.671875,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.359375,0.000000,0.0,0.0,...,18.1250,17.3125,16.5000,15.5625,14.6875,13.8125,12.5625,11.5625,10.6875,10.1250
138120,0,52.03125,-5.484375,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.296875,0.000000,0.0,0.0,...,15.6875,16.0625,16.5000,17.0625,17.6875,18.3750,18.1250,17.1250,17.5625,19.1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165072,34,57.84375,-2.109375,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.546875,0.124052,0.0,0.0,...,19.7500,19.5625,19.3750,19.1875,18.9375,18.6250,18.3125,17.3125,17.5000,19.3750
165073,34,57.84375,-1.828125,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.562500,0.013411,0.0,0.0,...,19.0000,18.8125,18.6250,18.5000,18.3750,18.2500,17.8750,16.8125,17.9375,20.3750
165074,34,57.84375,-1.546875,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.500000,0.026822,0.0,0.0,...,18.1875,18.0625,17.9375,17.8750,17.8750,17.8750,17.3125,16.3750,18.7500,21.8750
165075,34,57.84375,-1.265625,0 days 06:00:00,2021-10-20 18:00:00,2021-10-23,0.531250,0.013411,0.0,0.0,...,17.5000,17.3750,17.3750,17.4375,17.5625,17.5625,16.4375,16.1250,20.2500,24.1250


In [54]:
df2['time']

138116   2021-10-23
138117   2021-10-23
138118   2021-10-23
138119   2021-10-23
138120   2021-10-23
            ...    
165072   2021-10-23
165073   2021-10-23
165074   2021-10-23
165075   2021-10-23
165076   2021-10-23
Name: time, Length: 26961, dtype: datetime64[ns]

In [50]:
df2['time'].value_counts()

2021-10-23    26961
Name: time, dtype: int64

In [20]:
mg_df[mg_df['time'] == '2021-10-20 12:00:00']['realization'].value_counts()

0     1530
18    1530
33    1530
32    1530
31    1530
30    1530
29    1530
28    1530
27    1530
26    1530
25    1530
24    1530
23    1530
22    1530
21    1530
20    1530
19    1530
34    1530
Name: realization, dtype: int64

In [ ]:
radar_df = pandas.read_csv(radar_df_path)
radar_df

In [ ]:
radar_cubelist_regridded = iris.load(str(radar_nc_path))
radar_cubelist_regridded

In [ ]:
[c1 for c1 in radar_cubelist_regridded.extract_cube('radar_max_rain_instant').coord('time').cells()]

In [ ]:
[vt.timestamp() for vt in
             validity_times]

In [ ]:
target_cube = iris.load_cube('/project/informatics_lab/precip_rediagnosis/target_cube.nc')

In [ ]:
target_cube

In [ ]:
[datetime.datetime.fromtimestamp(vt1.timestamp()) for vt1 in validity_times]

In [ ]:
cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian')

In [ ]:
[c1 for c1 in iris.coords.DimCoord(
            [vt.timestamp() for vt in
             validity_times],
            var_name='time',
            units=target_cube.coord('time').units,
        ).cells()]

In [ ]:
[c1 for c1 in iris.coords.DimCoord(
            [vt.timestamp() for vt in
             validity_times],
            var_name='time',
            units=radar_cubelist_regridded.extract_cube('radar_max_rain_instant').coord('time').units,
        ).cells()]

In [ ]:
dt_list1 = [dt1 for dt1 in pandas.date_range(datetime.datetime(2020,8,3,12,0), datetime.datetime(2020,8,5,12,0), tz='UTC',freq=datetime.timedelta(hours=6)).to_pydatetime()]
dt_list1

In [ ]:
[c1 for c1 in iris.coords.DimCoord(
            [cf_units.encode_time(year=dt1.year,month=dt1.month, day=dt1.day, hour=dt1.hour, minute=dt1.minute, second=0) for dt1 in dt_list1],
            var_name='time',
            units=cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'),
        ).cells()]


In [ ]:
[c1 for c1 in iris.coords.DimCoord(
            [dt1.timestamp() for dt1 in dt_list1],
            var_name='time',
            units=cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'),
        ).cells()]


In [ ]:
[c1 for c1 in iris.coords.DimCoord(
            [dt1.timestamp() for dt1 in list(pandas.date_range(datetime.datetime(2020,8,3,12,0), datetime.datetime(2020,8,5,12,0), freq=datetime.timedelta(hours=6)).to_pydatetime())],
            var_name='time',
            units=cf_units.Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'),
        ).cells()]

In [ ]:
radar_cubelist_regridded.extract_cube('radar_max_rain_instant').coord('time').units

In [ ]:
radar_df_path

In [ ]:
radar_nc_raw = iris.load_cube(str(radar_dir / 'composite_rainfall_20211020.nc'))

In [ ]:
[c1 for c1 in radar_nc_raw.coord('time').cells()][:10]

In [ ]:
def compare_time(t1, t2):
    print f'{t1.hour} {t1.hour} {t1.hour} {t1.hour} {t1.minute}'
    is_match = (t1.year == t2.year) and  (t1.month == t2.month) and  (t1.day == t2.day) and  (t1.hour== t2.hour) and  (t1.minute == t2.minute)
    return is_match


In [ ]:
list(radar_nc_raw.coord('time')[0].cells())[0].point

In [ ]:
validity_times

In [ ]:
print(validity_times[3])
radar_single_ts = radar_nc_raw.extract(iris.Constraint(time=lambda cell1: compare_time(cell1.point, validity_times[3]) ))
radar_single_ts 

In [ ]:
xarray.DataArray.from_iris(radar_single_ts).to_dataframe()['time'].value_counts()

In [ ]:
radar_df = pandas.read_csv(radar_df_path)
mg_df = pandas.read_csv(mg_df_path)

In [ ]:
radar_df

In [ ]:
iris.coordvalidity_times[2].timestamp()

In [ ]:
radar_df.time.min(), radar_df.time.max()

In [ ]:
radar_df.time

In [ ]:
radar_df.time.value_counts()

In [ ]:
mg_df

In [ ]:
mg_df.time.min(), mg_df.time.max()

In [ ]:
mg_df.time.value_counts()

In [ ]:
1565*18

In [ ]:
pandas.merge(radar_df, mg_df, on=['time','latitude','longitude'])


In [ ]:
cl1 = iris.cube.CubeList([iris.load_cube(str(f1)) for f1 in radar_root.iterdir() if 'rainfall' in f1.stem])
iris.util.equalise_attributes(cl1)
cl1.concatenate_cube()


In [ ]:
[iris.load(str(p1), 'lwe_convective_snowfall_rate') for p1 in (dir1 / 'mogreps-g').iterdir() if '202002150000' in str(p1) and 'prd' in str(p1)]
# cl2 =iris.cube.CubeList()
# cl2

In [ ]:
xarray.DataArray.from_iris(cl2[0]).to_netcdf('/scratch/shaddad/test01.nc')

In [ ]:
iris.load('/scratch/shaddad/test01.nc')

In [ ]:
dir1 = pathlib.Path('/scratch/shaddad/precip_rediagnosis/train_202207/202002_storm_dennis')
fnames1 = [
    'mogreps-g/20200214T1800Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200215T0000Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200215T0600Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200215T1200Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200215T1800Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200216T0000Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200216T0600Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200216T1200Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200216T1800Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200217T0000Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200217T0600Z-PT0006H00M-rainfall_rate.nc',
    'mogreps-g/20200217T1200Z-PT0006H00M-rainfall_rate.nc',
]
paths1 = [dir1 / fn1 for fn1 in fnames1]
paths1

In [ ]:
cl1 = iris.cube.CubeList([iris.load_cube(str(p1)) for p1 in paths1])
cl1

In [ ]:
xarray.DataArray.from_iris(cl1[1]).to_dataframe()

In [ ]:
iris.util.equalise_attributes(cl1)

In [ ]:
for c1 in cl1:
    c1.remove_coord('forecast_reference_time')
    c1.remove_coord('forecast_period')
    

In [ ]:
iris.util.unify_time_units(cl1)

In [ ]:
[list(c1.coord('time').cells())[0].point for c1 in cl1]

In [ ]:
print(cl1)

In [ ]:
xarray.DataArray.from_iris(cl1.merge()[0]).to_dataframe().time

In [ ]:
cl1.merge()[1].coord('realization')